In [1]:
import findspark
findspark.init("/home/gorazda/spark-2.4.7-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("recomender_system_example").getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data = spark.read.csv("movielens_ratings.csv", inferSchema=True, header=True)

In [6]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [7]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [8]:
train_data, test_data = data.randomSplit([0.8, 0.2])

In [9]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")

In [10]:
model = als.fit(train_data)

In [11]:
predictions = model.transform(test_data)

In [12]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    27|  1.1166921|
|     31|   3.0|     8|   3.479063|
|     85|   4.0|     7|    4.09603|
|     65|   1.0|    28|   2.163022|
|     65|   1.0|    19| 0.44606605|
|     65|   5.0|    23|  2.7858627|
|     53|   1.0|     6|  2.2585726|
|     53|   5.0|    21|  1.1183687|
|     53|   3.0|    14|   4.543725|
|     78|   1.0|     4|  1.1784328|
|     78|   1.0|    24|  0.7813751|
|     81|   1.0|    15|  0.3657652|
|     28|   1.0|    17|  4.0996146|
|     28|   1.0|    10|  1.2213085|
|     76|   1.0|     1|  2.7017233|
|     76|   1.0|    20|   4.370585|
|     76|   1.0|    11|  3.0087984|
|     26|   1.0|    13| -1.5187637|
|     26|   1.0|    20|  2.4348998|
|     26|   1.0|     9|0.008858949|
+-------+------+------+-----------+
only showing top 20 rows



In [15]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [16]:
rmse = evaluator.evaluate(predictions)

In [17]:
rmse

2.2882634253462113

In [19]:
single_user = test_data.filter(test_data["userId"] == 11).select(["movieId", "userId"])

In [20]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     11|    11|
|     18|    11|
|     27|    11|
|     37|    11|
|     38|    11|
|     75|    11|
|     76|    11|
|     77|    11|
|     86|    11|
|     88|    11|
+-------+------+



In [21]:
recommendations = model.transform(single_user)

In [24]:
recommendations.orderBy("prediction", ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     37|    11|  5.5592804|
|     88|    11|  5.0812483|
|     38|    11|   3.405036|
|     76|    11|  3.0087984|
|     86|    11|  2.0860713|
|     27|    11| 0.87791264|
|     11|    11|  0.8420315|
|     77|    11| -0.2631781|
|     18|    11|-0.53142333|
|     75|    11| -1.7068878|
+-------+------+-----------+



In [25]:
data.filter(data["userId"] == 11).show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      0|   1.0|    11|
|      6|   2.0|    11|
|      9|   1.0|    11|
|     10|   1.0|    11|
|     11|   1.0|    11|
|     12|   1.0|    11|
|     13|   4.0|    11|
|     16|   1.0|    11|
|     18|   5.0|    11|
|     19|   4.0|    11|
|     20|   1.0|    11|
|     21|   1.0|    11|
|     22|   1.0|    11|
|     23|   5.0|    11|
|     25|   1.0|    11|
|     27|   5.0|    11|
|     30|   5.0|    11|
|     32|   5.0|    11|
|     35|   3.0|    11|
|     36|   2.0|    11|
+-------+------+------+
only showing top 20 rows

